# Extract Lemmatization from JSON: Extended Parser
The code in this notebook will parse [ORACC](http://oracc.org) `JSON` files to extract lemmatization data for one or more projects. The code shows how the word-by-word data structure can be reformatted to a line-by-line or document-by-document structure and discusses various other options.

The output of the Extended Parser contains text IDs, line IDs, lemmas, and (potentially) other data. The first few code blocks are identical with the Basic Parser.

In [1]:
import pandas as pd
import zipfile
import json
import tqdm
import requests
import errno
import os

## 0 Create Directories, if Necessary
The two directories needed for this script are `jsonzip` and `output`. If they do not exist they are created, else: do nothing.

For the code, see [Stack Overflow](http://stackoverflow.com/questions/18973418/os-mkdirpath-returns-oserror-when-directory-does-not-exist).

In [2]:
directories = ['jsonzip', 'output']
for d in directories:
    try:
        os.mkdir(d)
    except OSError as exc:
        if exc.errno !=errno.EEXIST:
            raise
        pass

## 1.1 Input Project Names
Provide a list of one or more project names, separated by commas. Note that subprojects must be listed separately, they are not included in the main project. For instance:

`saao/saa01,saao/saa02,blms`

In [3]:
projects = input('Project(s): ').lower()

Project(s): dcclt


## 1.2 Split the List of Projects
Split the list of projects and create a list of project names.

In [4]:
p = projects.split(',')               # split at each comma and make a list called `p`
p = [x.strip() for x in p]        # strip spaces left and right of each entry in `p`

## 1.3 Download the ZIP files
For each project in the list download all the `json` files from `http://build-oracc.museum.upenn.edu/json/`. The file is called `PROJECT.zip` (for instance: `dcclt.zip`). For subprojects the file is called `PROJECT-SUBPROJECT.zip` (for instance `cams-gkab.zip`). 

For larger projects (such as [DCCLT](http://oracc.org/dcclt)) the `zip` file may be 25Mb or more. Downloading may take some time and it may be necessary to chunk the downloading process. The `iter_content()` function in the `requests` library takes care of that.

If you have downloaded the files by hand (and put them in the `jsonzip` directory) you may skip this cell and jump directly to section [2.1 The Parsejson() function](#head21).

In [5]:
CHUNK = 16 * 1024
for project in p:
    project = project.replace('/', '-')
    url = "http://build-oracc.museum.upenn.edu/json/" + project + ".zip"
    file = 'jsonzip/' + project + '.zip'
    with requests.get(url, stream=True) as r: 
        if r.status_code == 200:
            print("Downloading " + url + " saving as " + file)
            with open(file, 'wb') as f:
                for c in tqdm.tqdm(r.iter_content(chunk_size=CHUNK)):
                    f.write(c)
        else:
            print(url + " does not exist.")

3547it [00:15, 230.72it/s]


## <a name="head21"></a>2.1 The `parsejson()` function
The `parsejson()` function is essentially identical with that function in `First_JSON_parser.ipynb`, but it fetches more data. The field `word_id` consists of three parts, namely a text ID, line ID, and word ID, in the format `Q000039.76.2` meaning: the second word in line 76 of text object `Q000039`. Note that `76` is not a line number strictly speaking but an object reference within the text object. Things like horizontal rulings, columns, and breaks also get object references. The `word_id` field allows us to put lines, breaks, and horizontal drawings together in the proper order.

The field `label` is a human-legible label that refers a line or another part of the text; it may look like `o i 23` (obverse column 1 line 23) or `r v 23'` (reverse column 5 line 23 prime). The `label` field is used in online [ORACC](http://oracc.org) editions to indicate line numbers.

The fields `extent`, `scope`, and `state` give metatextual data about the condition of the object; they capture the number of broken lines or columns and similar information. 


In [6]:
def parsejson(text):
    for JSONobject in text["cdl"]:
        if "cdl" in JSONobject: 
            parsejson(JSONobject)
        if "label" in JSONobject:
            meta_d["label"] = JSONobject['label']
        if "type" in JSONobject and JSONobject["type"] == "field-start": # this is for sign lists, identifying fields such as
            meta_d["fieldtype"] = JSONobject["subtype"]                    # sign, pronunciation, translation.
        if "type" in JSONobject and JSONobject["type"] == "field-end":
            meta_d.pop("field", None)                           # remove the key "subtype" to prevent it from being copied 
                                                              # to all subsequent lemmas (which may not have fields)
        if "f" in JSONobject:
            lemma = JSONobject["f"]
            lemma["id_word"] = JSONobject["ref"]
            lemma['label'] = meta_d["label"]
            lemma["id_text"] = meta_d["id_text"]
            if "fieldtype" in meta_d:
                lemma["field"] = meta_d["fieldtype"]
            lemm_l.append(lemma)
        if "strict" in JSONobject and JSONobject["strict"] == "1":
            lemma = {key: JSONobject[key] for key in dollar_keys}
            lemma["id_word"] = JSONobject["ref"]
            lemma["id_text"] = meta_d["id_text"]
            lemm_l.append(lemma)
    return

## 2.2 Call the `parsejson()` function for every `JSON` file
The code in this cell will iterate through the list of projects entered above (1.1). For each project the `JSON` zip file is located in the directory `jsonzip`, named PROJECT.zip. The `zip` file contains a directory that is called `corpusjson` that contains a JSON file for every text that is available in that corpus. The files are called after their text IDs in the pattern `P######.json` (or `Q######.json` or `X######.json`).

The function `namelist()` of the `zipfile` package is used to create a list of the names of all the files in the ZIP. From this list we select all the file names in the `corpusjson` directory with extension `.json` (this way we exclude the name of the directory itself). 

Each of these files is read from the `zip` file and loaded with the command `json.loads()`, which transforms the string into a proper JSON object. 

This JSON object (essentially a Python dictionary), which is called `data_json` is now sent to the `parsejson()` function. The function adds lemmata to the `lemm_l` list. In the end, `lemm_l` will contain as many list elements as there are words in all the texts in the projects requested.

The dictionary `meta_d` is created to hold temporary information. The value of the key `id_text` is updated in the main process every time a new JSON file is opened and send to the `parsejson()` function. The `parsejson()` function itself will change values or add new keys, depending on the information found while iterating through the JSON file. When a new lemma row is created, `parsejon()` will supply data such as `id_text`, `label` and (potentially) other information from `meta_d`.

In [7]:
lemm_l = []
meta_d = {"label": None, "id_text": None}
dollar_keys = ["extent", "scope", "state"]
for project in p:
    print("Parsing " + project)
    file = "jsonzip/" + project.replace("/", "-") + ".zip"
    try:
        z = zipfile.ZipFile(file)       # create a Zipfile object
    except:
        print(file + " does not exist or is not a proper ZIP file")
        continue
    files = z.namelist()     # list of all the files in the ZIP
    files = [name for name in files if "corpusjson" in name and name[-5:] == '.json']                                                                                                  #that holds all the P, Q, and X numbers.
    for filename in tqdm.tqdm(files):                            #iterate over the file names
        id_text = project + filename[-13:-5] # id_text is, for instance, blms/P414332
        meta_d["id_text"] = id_text
        try:
            st = z.read(filename).decode('utf-8')         #read and decode the json file of one particular text
            data_json = json.loads(st)                # make it into a json object (essentially a dictionary)
            parsejson(data_json)               # and send to the parsejson() function
        except:
            print(id_text + ' is not available or not complete')
    z.close()

Parsing dcclt


100%|█████████████████████████████████████| 4496/4496 [00:30<00:00, 147.97it/s]


## 3 Data Structuring
### 3.1 Transform the Data into a DataFrame
The list `lemm_l` is transformed into a `pandas` dataframe for further manipulation.

For various reasons not all JSON files will have all data types that potentially exist in an [ORACC](http://oracc.org) signature. Only Sumerian words have a `base`, so if your data set has no Sumerian, this column will not exist in the DataFrame.  If a text has no breakage information in the form of `1 line broken` (etc.) the fields `extent`, `scope`, and `state` do not exist. Where such fields are referenced below, the code may fail and you may need to adjust some lines.

In [8]:
words_df = pd.DataFrame(lemm_l)
words_df = words_df.fillna('')   # replace NaN (Not a Number) with empty string
words_df

,base,cf,cont,delim,epos,extent,field,form,gdl,gw,...,id_word,label,lang,morph,norm,norm0,pos,scope,sense,state
0,,,,,,,,kuš,"[{'v': 'kuš', 'id': 'P231172.4.1.0', 'break': ...",,...,P231172.4.1,o ii 1,sux,,,,,,,
1,,,,,,,,x,"[{'x': 'ellipsis', 'id': 'P231172.4.2.0', 'bre...",,...,P231172.4.2,o ii 1,sux,,,,,,,
2,,,,,,,,kuš,"[{'v': 'kuš', 'id': 'P231172.5.1.0'}]",,...,P231172.5.1,o ii 2,sux,,,,,,,
3,,,,,,,,x,"[{'x': 'ellipsis', 'id': 'P231172.5.2.0', 'bre...",,...,P231172.5.2,o ii 2,sux,,,,,,,
4,,,,,,,,kuš,"[{'v': 'kuš', 'id': 'P231172.6.1.0'}]",,...,P231172.6.1,o ii 3,sux,,,,,,,
5,,,,,,,,x,"[{'x': 'ellipsis', 'id': 'P231172.6.2.0', 'bre...",,...,P231172.6.2,o ii 3,sux,,,,,,,
6,,,,,,,,kuš,"[{'v': 'kuš', 'id': 'P231172.7.1.0'}]",,...,P231172.7.1,o ii 4,sux,,,,,,,
7,,,,,,,,x,"[{'x': 'ellipsis', 'id': 'P231172.7.2.0', 'bre...",,...,P231172.7.2,o ii 4,sux,,,,,,,
8,,,,,,,,kuš,"[{'v': 'kuš', 'id': 'P231172.8.1.0'}]",,...,P231172.8.1,o ii 5,sux,,,,,,,
9,,,,,,,,x,"[{'x': 'ellipsis', 'id': 'P231172.8.2.0', 'bre...",,...,P231172.8.2,o ii 5,sux,,,,,,,


## 3.2 Remove Spaces and Commas from Guide Word and Sense
Spaces and commas in Guide Word and Sense may cause trouble in computational methods in tokenization, or when saved in Comma Separated Values format. All spaces and commas are replaced by hyphens and nothing (empty string), respectively. By default the `replace()` function in `pandas` will match the entire string (that is, "lugal" matches "lugal" but there is no match between "l" and "lugal"). In order to match partial strings the parameter `regex` must be set to `True`.

The `replace()` function takes a nested dictionary as argument. The top-level keys identify the columns on which the `replace()` function should operate (in this case 'gw' and 'sense'). The value of each key is another dictionary with the search string as key and the replace string as value.

In [9]:
findreplace = {' ' : '-', ',' : ''}
words_df = words_df.replace({'gw' : findreplace, 'sense' : findreplace}, regex=True)

The columns in the resulting DataFrame correspond to the elements of a full [ORACC](http://oracc.org) signature, plus information about text, line, and word ids:
* base (Sumerian only)
* cf (Citation Form)
* cont (continuation of the base; Sumerian only)
* epos (Effective Part of Speech)
* form (transliteration, omitting all flags such as indication of breakage)
* frag (transliteration; including flags)
* gdl_utf8 (cuneiform)
* gw (Guide Word: main or first translation in standard dictionary)
* id_text (six-digit P, Q, or X number)
* id_word (word ID in the format Text_ID.Line_ID.Word_ID)
* label (traditional line number in the form o ii 2' (obverse column 2 line 2'), etc.)
* lang (language code, including sux, sux-x-emegir, sux-x-emesal, akk, akk-x-stdbab, etc)
* morph (Morphology; Sumerian only)
* norm (Normalization: Akkadian)
* norm0 (Normalization: Sumerian)
* pos (Part of Speech)
* sense (contextual meaning)
* sig (full ORACC signature)

Not all data elements (columns) are available for all words. Sumerian words never have a `norm`, Akkadian words do not have `norm0`, `base`, `cont`, or `morph`. Most data elements are only present when the word is lemmatized; only `lang`, `form`, `id_word`, and `id_text` should always be there.

## Create Line ID
The DataFrame currently has a word-by-word data representation. We will add to each word a field `id_line` that will make it possible to reconstruct lines. This newly created field `id_line` is different from a traditional line number (found in the field "label") in two ways. First, id_line is an integer, so that lines are sorted correctly. Second, `id_line` is assigned to words, but also to gaps and horizontal drawings on the tablet. The field `id_line` will allow us to keep all these elements in their proper order.

The field "id_line" is created by splitting the field "id_word" into (two or) three elements. The format of "id_word" is IDtext.line.word. The middle part, id_line, is selected and its data type is changed from string to integer. Rows that represent gaps in the text or horizontal drawings have an "id_word" in the format ID.text.line, but are treated in exactly the same way.

In [10]:
words_df['id_line'] = [int(wordid.split('.')[1]) for wordid in words_df['id_word']]

## 4 Save Results in CSV file or in Pickle
The output file is called `parsed.csv` and is placed in the directory `output`. In most computers, `csv` files open automatically in Excel. This program does not deal well with `utf-8` encoding (files in `utf-8` need to be imported; see the instructions [here](https://www.itg.ias.edu/content/how-import-csv-file-uses-utf-8-character-encoding-0). If you intend to use the file in Excel, change `encoding ='utf-8'` to `encoding='utf-16'`. For usage in computational text analysis applications `utf-8` is usually preferred. 

The Pandas function `to_pickle()` writes a binary file that can be opened in a later phase of the project with the `read_pickle()` command and will reproduce exactly the same DataFrame with the same data structure. The resulting file cannot be used in other programs.

In [11]:
savefile =  'parsed.csv'
with open('output/' + savefile, 'w', encoding="utf-8") as w:
    words_df.to_csv(w, index=False)
pickled = "parsed.p"
with open('output/' + pickled, 'wb') as w:
    words_df.to_pickle(w)

# 5 Post Processing
# 5.1 Manipulate for Analysis on Line level
For analyses that use a line as unit of analysis (e.g. lines in lexical texts as used in Chapter 3) one may need to create lemmas and combine these into lines by using the `id_line` variable.

## 5.1.1 Create Lemma Column
A lemma, [ORACC](http://oracc.org) style, combines Citation Form, GuideWord and POS into a unique reference to one particular lemma in a standard dictionary, as in `lugal[king]N` (Sumerian) or `šarru[king]N`. Usually, not all words in a text are lemmatized, because a word may be (partly) broken and/or unknown. Unlemmatized and unlemmatizable words will receive a place-holder lemmatization that consists of the transliteration of the word (instead of the Citation Form), with `NA` as GuideWord and `NA` as POS, as in `i-bu-x[NA]NA`. Note that `NA` is a string. Finally, rows representing horizontal rulings or broken lines have the empty string in both Citation Form and Form. In those cases Lemma should have the empty string, too.

In [12]:
words_df['lemma'] = words_df["cf"] + '[' + words_df["gw"] + ']' + words_df["pos"]
words_df.loc[words_df["cf"] == "" , 'lemma'] = words_df['form'] + '[NA]NA'
words_df.loc[words_df["form"] == "", 'lemma'] = ""
words_df

,base,cf,cont,delim,epos,extent,field,form,gdl,gw,...,lang,morph,norm,norm0,pos,scope,sense,state,id_line,lemma
0,,,,,,,,kuš,"[{'v': 'kuš', 'id': 'P231172.4.1.0', 'break': ...",,...,sux,,,,,,,,4,kuš[NA]NA
1,,,,,,,,x,"[{'x': 'ellipsis', 'id': 'P231172.4.2.0', 'bre...",,...,sux,,,,,,,,4,x[NA]NA
2,,,,,,,,kuš,"[{'v': 'kuš', 'id': 'P231172.5.1.0'}]",,...,sux,,,,,,,,5,kuš[NA]NA
3,,,,,,,,x,"[{'x': 'ellipsis', 'id': 'P231172.5.2.0', 'bre...",,...,sux,,,,,,,,5,x[NA]NA
4,,,,,,,,kuš,"[{'v': 'kuš', 'id': 'P231172.6.1.0'}]",,...,sux,,,,,,,,6,kuš[NA]NA
5,,,,,,,,x,"[{'x': 'ellipsis', 'id': 'P231172.6.2.0', 'bre...",,...,sux,,,,,,,,6,x[NA]NA
6,,,,,,,,kuš,"[{'v': 'kuš', 'id': 'P231172.7.1.0'}]",,...,sux,,,,,,,,7,kuš[NA]NA
7,,,,,,,,x,"[{'x': 'ellipsis', 'id': 'P231172.7.2.0', 'bre...",,...,sux,,,,,,,,7,x[NA]NA
8,,,,,,,,kuš,"[{'v': 'kuš', 'id': 'P231172.8.1.0'}]",,...,sux,,,,,,,,8,kuš[NA]NA
9,,,,,,,,x,"[{'x': 'ellipsis', 'id': 'P231172.8.2.0', 'bre...",,...,sux,,,,,,,,8,x[NA]NA


## 5.1.2 Group by Line
In the `words_df` dataframe each word has a separate row. In order to change this into a line-by-line representation we use the `pandas` `groupby()` function, using `id_text`, `id_line` and `label` fields as the sorting arguments. 

The fields that are aggregated are `lemma`, `extent`, `scope`, and `state`. The fields `extent`, `scope`, and `state` represent data on the number of broken lines. For instance, the notation `4 lines missing` in the [ORACC](http://oracc.org) edition will result in `extent = "4"`, `scope = "line"`, `state = "missing"` (note that the value of `extent` is a string and will be `"n"` if the number of missing lines or columns is unknown).

If your data does not have the fields `extent`, `scope`, and `state` the code below will fail - simply delete the lines that reference those fields.

In [13]:
lines = words_df.groupby([words_df['id_text'], words_df['id_line'], words_df['label']]).agg({
        'lemma': ' '.join,
        'extent': ''.join, 
        'scope': ''.join,
        'state': ''.join
    }).reset_index()
lines

,id_text,id_line,label,lemma,extent,scope,state
0,dcclt/P000001,3,o i 1,1(N01)[NA]NA x[NA]NA,,,
1,dcclt/P000001,4,o i 2,1(N01)[NA]NA ABGAL[NA]NA,,,
2,dcclt/P000001,5,o i 3,1(N01)[NA]NA KINGAL[NA]NA,,,
3,dcclt/P000001,7,o ii 1,1(N01)[NA]NA x[NA]NA,,,
4,dcclt/P000001,8,o ii 2,1(N01)[NA]NA GAL[NA]NA UMUN₂[NA]NA,,,
5,dcclt/P000001,9,o ii 3,1(N01)[NA]NA GAL[NA]NA UMUN₂[NA]NA KU₃[NA]NA,,,
6,dcclt/P000001,11,o iii 1,1(N01)[NA]NA DUB[NA]NA SANGA[NA]NA,,,
7,dcclt/P000001,12,o iii 2,1(N01)[NA]NA SUG₅[NA]NA SAG[NA]NA,,,
8,dcclt/P000001,13,o iii 3,1(N01)[NA]NA UB[NA]NA SAG[NA]NA,,,
9,dcclt/P000002,3,o i 1,1(N01)[NA]NA x[NA]NA,,,


## 5.2 Alternative: Texts in Normalized Transcription
This code (which is useful mostly for Akkadian texts) will produce a text in normalized transcription, essentially following the pattern of the preceding. Before grouping words into documents, we need to take care of words that have not been normalized (for instance because of breakage), using the field `form`. The field `norm1` now has the normalized form of the word if it is available; if not it has the raw transliteration (without flags or breakage information).

In [14]:
words_df["norm1"] = words_df["norm"]
words_df.loc[words_df["norm1"] == "" , 'norm1'] = words_df['form']

In [15]:
texts_norm = words_df.groupby([words_df['id_text']]).agg({
        'norm1': ' '.join,
    }).reset_index()
texts_norm

,id_text,norm1
0,dcclt/P000001,1(N01) x 1(N01) ABGAL 1(N01) KINGAL 1(N01) x 1...
1,dcclt/P000002,1(N01) x 1(N01) GAL ŠITA 1(N01) ABGAL 1(N01) x...
2,dcclt/P000003,1(N01) NI 1(N01) |NI×X| 1(N01) IR 1(N01) BIR 1...
3,dcclt/P000004,1(N01) |ŠE.NAM₂| 1(N01) PA x 1(N01) x 1(N01) x
4,dcclt/P000005,1(N01) EŠDA 1(N01) KAB 1(N01) DI@t 1(N01) EN 1...
5,dcclt/P000006,1(N01) x 1(N01) NAMEŠDA 1(N01) KAB NAM₂ 1(N01)...
6,dcclt/P000007,1(N01) X x 1(N01) |ŠE.NAM₂| 1(N01) PAP 1(N01) ...
7,dcclt/P000008,1(N01) SILA₃ IŠ x 1(N01) NI TUR 1(N01) SAL ZAT...
8,dcclt/P000009,1(N01) GAL x 1(N01) GAL GURUŠDA 1(N01) SANGA G...
9,dcclt/P000010,1(N01) x EN NUN 1(N01) X EN AMAR 1(N01) X |ŠU₂...


### 5.2.1 Save Normalized Transcriptions
The `texts_norm` DataFrame has one complete document in normalized transcription in each row. The code below saves each row as a separate `.txt` file, named after the document's ID.

In [16]:
for idx, Q in enumerate(texts_norm["id_text"]):
    savefile =  Q[-7:] + '.txt'
    with open('output/' + savefile, 'w', encoding="utf-8") as w:
        texts_norm.iloc[idx].to_csv(w, index = False)